# Dynamic Attributes and Properties
Data attributes and methods are collectively known as attributes in Python: a method is just an attribute that is callable. Besides data attributes and methods, we can also create properties, which can be used to replace a public data attribute with accessor methods (i.e., getter/setter), without changing the class interface. This agrees with the Uniform access principle: All services offered by a module should be available through a uniform notation, which does not betray whether they are implemented through storage or through computation.

Besides properties, Python provides a rich API for controlling attribute access and implementing dynamic attributes. The interpreter calls special methods such as __getattr__ and __setattr__ to evaluate attribute access using dot notation.

In [7]:
class A:
    def __getattr__(self, attr):
        ''' __getattr__ is called when the default attribute access fails with an AttributeError.
            It lets you define an optional value to return if the attribute doesn't exist.
            Similar to dict.get()
        '''
        print("Getting attribute ", attr)
        return 42


In [8]:
a = A()
a.x

Getting attribute  x


42

In [10]:
d = {"a": 1, "b":2}
d.get("c", 42)

42

In [9]:
class A:
    def __init__(self):
        self.x = 7

    def __setattr__(self, attr, value):
        ''' Called when an attribute assignment is attempted. '''
        print("Setting attribute ", attr, " to ", value)
        self.__dict__[attr] = value


In [10]:
a = A()  # setattr called on init

Setting attribute  x  to  7


In [11]:
a.x

7

In [12]:
a.x = 9  # also when we set here

Setting attribute  x  to  9


We could stop someone from setting attributes that begin with an underscore...

In [13]:
class A:
    def __init__(self):
        self._x = 7
        self._y = 8
        
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y
    
    def __setattr__(self, attr, value):
        if attr in self.__dict__:
            if attr.startswith("_"):
                print("Nah bro!")
                return
        self.__dict__[attr] = value


In [14]:
a = A()
a.x, a.y

(7, 8)

In [15]:
a._x = 12

Nah bro!


In [16]:
a.x

7

## Data Wrangling with Dynamic Attributes

In [18]:
d = { "Schedule":
 { "conferences": [{"serial": 115 }],
 "events": [
 { "serial": 34505,
 "name": "Why Schools Don´t Use Open Source to Teach Programming",
 "event_type": "40-minute conference session",
 "time_start": "2014-07-23 11:30:00",
 "time_stop": "2014-07-23 12:10:00",
 "venue_serial": 1462,
 "description": "Aside from the fact that high school programming...",
 "website_url": "http://oscon.com/oscon2014/public/schedule/detail/34505",
 "speakers": [157509],
 "categories": ["Education"] }
 ],
 "speakers": [
 { "serial": 157509,
 "name": "Robert Lefkowitz",
 "photo": None,
 "url": "http://sharewave.com/",
 "position": "CTO",
 "affiliation": "Sharewave",
 "twitter": "sharewaveteam",
 "bio": "Robert ´r0ml´ Lefkowitz is the CTO at Sharewave, a startup..." }
 ],
 "venues": [
 { "serial": 1462,
 "name": "F151",
 "category": "Conference Venues" }
 ]
 }
}


In [19]:
d

{'Schedule': {'conferences': [{'serial': 115}],
  'events': [{'serial': 34505,
    'name': 'Why Schools Don´t Use Open Source to Teach Programming',
    'event_type': '40-minute conference session',
    'time_start': '2014-07-23 11:30:00',
    'time_stop': '2014-07-23 12:10:00',
    'venue_serial': 1462,
    'description': 'Aside from the fact that high school programming...',
    'website_url': 'http://oscon.com/oscon2014/public/schedule/detail/34505',
    'speakers': [157509],
    'categories': ['Education']}],
  'speakers': [{'serial': 157509,
    'name': 'Robert Lefkowitz',
    'photo': None,
    'url': 'http://sharewave.com/',
    'position': 'CTO',
    'affiliation': 'Sharewave',
    'twitter': 'sharewaveteam',
    'bio': 'Robert ´r0ml´ Lefkowitz is the CTO at Sharewave, a startup...'}],
  'venues': [{'serial': 1462,
    'name': 'F151',
    'category': 'Conference Venues'}]}}

In [3]:
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'

if os.path.isfile(JSON):
    os.remove(JSON)

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)
        with urlopen(URL) as remote, open(JSON, 'wb') as local:
            return remote.read()


In [4]:
feed = load()

C:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:15: UserWarning: downloading http://www.oreilly.com/pub/sc/osconfeed to data/osconfeed.json
  from ipykernel import kernelapp as app


In [7]:
len(feed)

787465

## Using a Property for Attribute Validation

In [1]:
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price


In [2]:
# there are issues with this
raisins = LineItem("Golden Raisins", 10, 6.95)
raisins.subtotal()

69.5

In [3]:
raisins.weight = -20  # garbage in 
raisins.subtotal()    # and garbage out

-139.0

Beyond allowing for a read only property with the @property decorator, we can incorporate validation.

In [4]:
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    @property
    def weight(self):
        return self.__weight
    
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError("value must be > 0")


In [8]:
import traceback

try:
    walnuts = LineItem("walnuts", 0, 10.00)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-8-16d58f23fbb2>", line 4, in <module>
    walnuts = LineItem("walnuts", 0, 10.00)
  File "<ipython-input-4-fc91de10028d>", line 4, in __init__
    self.weight = weight
  File "<ipython-input-4-fc91de10028d>", line 19, in weight
    raise ValueError("value must be > 0")
ValueError: value must be > 0


## A Proper Look at Properties
Although often used as a decorator, the property built-in is actually a class. This is the full signature of the property constructor: property(fget=None, fset=None, fdel=None, doc=None). The property type was added in Python 2.2, but the @ decorator syntax appeared only in Python 2.4. Here is the 'classic' syntax for defining properties:

In [10]:
# A plain getter.
# A plain setter
# Build the property and assign it to a public class attribute.

class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
 
    def subtotal(self):
        return self.weight * self.price

    def get_weight(self):
        return self.__weight

    def set_weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

    weight = property(get_weight, set_weight) 


In [11]:
item = LineItem("toy", 2.5, 100)

In [13]:
item.weight

2.5

In [15]:
import traceback

try:
    item.weight = 0
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-15-f23604991c7c>", line 4, in <module>
    item.weight = 0
  File "<ipython-input-10-77af64ae02ec>", line 21, in set_weight
    raise ValueError('value must be > 0')
ValueError: value must be > 0


## Properties Override Instance Attributes
Properties are always class attributes, but they actually manage attribute access in the
instances of the class.

In [19]:
class X:
    a = 3
    def __init__(self):
        self.a = 7
        
    @property
    def b(self):
        return 10


In [20]:
x = X()
x.a  # the instance attribute overrides in these cases

7

In [21]:
x.b

10

In [22]:
x.b = 11  # we have no setter in this case

AttributeError: can't set attribute

Here we cannot override the property attribute as it has no setter... it's effectively read only.

The main point of this section is that an expression like obj.attr does not search for attr starting with obj. The search actually starts at obj.__class__, and only if there is no property named attr in the class, Python looks in the obj instance itself.

In [24]:
x.__dict__

{'a': 7}

In [23]:
X.__dict__

mappingproxy({'__module__': '__main__',
              'a': 3,
              '__init__': <function __main__.X.__init__(self)>,
              'b': <property at 0x1a537d1f908>,
              '__dict__': <attribute '__dict__' of 'X' objects>,
              '__weakref__': <attribute '__weakref__' of 'X' objects>,
              '__doc__': None})

In [25]:
X.b

In [29]:
X.b.fget(x)  # recall signature property(fget=None, fset=None, fdel=None, doc=None)

10

## Coding a Property Factory

In [1]:
def quantity(storage_name):

    def qty_getter(instance):
        return instance.__dict__[storage_name]

    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
        return property(qty_getter, qty_setter) 


In [2]:
class LineItem:
    weight = quantity('weight')
    price = quantity('price') 

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price


In [3]:
item = LineItem("toy", 2.5, 100)

In [4]:
item.weight

2.5

In [5]:
LineItem.__dict__

mappingproxy({'__module__': '__main__',
              'weight': None,
              'price': None,
              '__init__': <function __main__.LineItem.__init__(self, description, weight, price)>,
              'subtotal': <function __main__.LineItem.subtotal(self)>,
              '__dict__': <attribute '__dict__' of 'LineItem' objects>,
              '__weakref__': <attribute '__weakref__' of 'LineItem' objects>,
              '__doc__': None})

In [6]:
item.__dict__

{'description': 'toy', 'weight': 2.5, 'price': 100}

In [8]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
nutmeg.weight, nutmeg.price

(8, 13.95)

In [9]:
sorted(vars(nutmeg).items()) 

[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]

## Handling Attribute Deletion
A silly example to demonstrate the @my_property.deleter decorator.

In [11]:
class BlackKnight:
    def __init__(self):

        self.members = ['an arm',
                        'another arm',
                        'a leg',
                        'another leg']

        self.phrases = ["'Tis but a scratch.",
                        "It's just a flesh wound.",
                        "I'm invincible!",
                        "All right, we'll call it a draw."]

    @property
    def member(self):
        print('next member is:')
        return self.members[0]
 
    @member.deleter
    def member(self):
        text = 'BLACK KNIGHT (loses {})\n-- {}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))


In [12]:
knight = BlackKnight()

In [13]:
knight.member

next member is:


'an arm'

In [14]:
del knight.member

BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch.


In [15]:
knight.members

['another arm', 'a leg', 'another leg']

In [16]:
knight.phrases

["It's just a flesh wound.",
 "I'm invincible!",
 "All right, we'll call it a draw."]

In [17]:
del knight.member

BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.


In [18]:
del knight.member

BLACK KNIGHT (loses a leg)
-- I'm invincible!


In [19]:
knight.members, knight.phrases

(['another leg'], ["All right, we'll call it a draw."])

The fdel argument is used to set the deleter function.

***